# Baseline Run

Without any experimental tweaks

In [4]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v5-selective-loss"
DEEPSPEED_STRAT="deepspeed_stage_1"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v5-exp/selective-loss
TRAINER_DIR: /home/recursal/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /home/recursal/RWKV-infctx-trainer


In [3]:
# Lets initialized the L12-D768 model with the init_model.py code
!cd "{TRAINER_DIR}" && python3 init_model.py \
    --n_layer 12 --n_embd 768 \
    --vocab_size world \
    --skip-if-exists --safe-init \
    ../model/L12-D768-world-v5-init.pth

[2024-01-18 09:25:17,072] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.2'
---- Initializing model ----
No of layers: 12
Embedding size: 768
Output model path: ../model/L12-D768-world-v5-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Model exists, skipping init_model


In [4]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-16k-packing.yaml"

Map (num_proc=160):  60%|██▍ | 600000/1000000 [00:06<00:04, 88429.46 examples/s]
Filter (num_proc=160): 100%|█| 1000000/1000000 [00:05<00:00, 182033.90 examples/
Map (num_proc=160): 100%|█████| 120800/120800 [00:05<00:00, 20558.33 examples/s]
Saving the dataset (4/4 shards): 100%|█| 18147/18147 [00:03<00:00, 5766.48 examp
Saving the dataset (1/1 shards): 100%|█| 13423/13423 [00:00<00:00, 33568.34 exam


# Multi-epoch training

In [ ]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-16k-packing.yaml" \
        --model.load_model="../model/L12-D768-world-v5-init.pth" \
        --data.skip_datapath_setup=True \
        --trainer.callbacks.init_args.dirpath="../checkpoint/selective-loss/baseline/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Baseline (trainsize=4k,packsize=16k) - ({DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.target_batch_size=128 \
        --trainer.microbatch_size=4 \
        --trainer.devices="{GPU_DEVICES}"

[2024-01-18 09:45:10,134] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v5-exp/selective-loss/config/enwiki_100k-world-16k-packing.yaml', '--model.load_model=../model/L12-D768-world-v5-init.pth', '--data.skip_datapath_setup=True', '--trainer.callbacks.init_args.dirpath=../checkpoint/selective-loss/baseline/', '--trainer.logger.init_args.name=infctx-v5-selective-loss - Baseline (trainsize=4k,packsize=16k) - (deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.target_batch